In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse 
SRC_PATH = '/home/ec2-user/SageMaker/mastering-ml-on-aws/chapter2/'


In [4]:
stop_words = [word.strip() for word in open(SRC_PATH + 'stop_words.txt').readlines()]

with open(SRC_PATH + 'dem.txt', 'r') as file:
    dem_text = [line.strip('\n') for line in file]

with open(SRC_PATH + 'gop.txt', 'r') as file:
    gop_text = [line.strip('\n') for line in file]

vectorizer = CountVectorizer(input=dem_text + gop_text,
                             stop_words=stop_words,
                             max_features=1200)
dem_bow = vectorizer.fit_transform(dem_text)
gop_bow = vectorizer.fit_transform(gop_text)


FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/mastering-ml-on-aws/chapter2/dem.txt'

In [5]:
(dem_bow.shape, gop_bow.shape)


((200, 1200), (200, 1200))

In [6]:
x = sparse.vstack((dem_bow, gop_bow))
ones = np.ones(200)
zeros = np.zeros(200)
y = np.hstack((ones, zeros))

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [8]:
from sklearn.naive_bayes import BernoulliNB
naive_bayes = BernoulliNB()
model = naive_bayes.fit(x_train, y_train)


In [9]:
y_predictions = model.predict(x_test)

In [10]:
y_predictions, y_test

(array([0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.,
        0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.]),
 array([0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
        0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.]))

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predictions)

0.95

In [12]:
# show also how to work with pipelines
from sklearn.pipeline import Pipeline

x_train, x_test, y_train, y_test = train_test_split(dem_text + gop_text, y, test_size=0.25, random_state=5)
pipeline = Pipeline([('vect', vectorizer), ('nb', naive_bayes)])
pipeline_model = pipeline.fit(x_train, y_train)
y_predictions = pipeline_model.predict(x_test)
accuracy_score(y_test, y_predictions)

0.92